In [58]:
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np


def encode_message(bits, bases, n):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message



def measure_message(message, bases, n):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        result = aer_sim.run(message[q], shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

def remove_garbage(a_bases, b_bases, bits, n):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

def sample_bits(message, bits, selection, print):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        if (print):
            display(message[i].draw())
        sample.append(bits.pop(i))
    return sample

def run(seed, n, sample_size, display_bit, eve):
    np.random.seed(seed=seed)

    ## Step 1
    # Alice generates bits
    alice_bits = randint(2, size=n)

    ## Step 2
    # Create an array to tell us which qubits
    # are encoded in which bases
    alice_bases = randint(2, size=n)
    message = encode_message(alice_bits, alice_bases, n)
    print("Initial message qubit")
    display(message[display_bit].draw())
    ## Interception!!
    if (eve):
        eve_bases = randint(2, size=n)
        intercepted_message = measure_message(message, eve_bases, n)
        print("After Eve's measurement")
        display(message[display_bit].draw())
    ## Step 3
    # Decide which basis to measure in:
    bob_bases = randint(2, size=n)
    bob_results = measure_message(message, bob_bases, n)
    print("After Bob's measurement")
    display(message[display_bit].draw())
    ## Step 4
    alice_key = remove_garbage(alice_bases, bob_bases, alice_bits, n)
    bob_key = remove_garbage(alice_bases, bob_bases, bob_results, n)
    print("alice_key = ",alice_key)
    print("bob_key = ", bob_key)

    ## Step 5
    bit_selection = randint(n, size=sample_size)

    bob_sample = sample_bits(message, bob_key, bit_selection , 1)
    alice_sample = sample_bits(message, alice_key, bit_selection, 0)
    print("alice_sample = "+ str(alice_sample))
    print("  bob_sample = " + str(bob_sample))
    print("Interception detected: ", bob_sample != alice_sample)
    


In [62]:

seed = randint(9949)
size = 100
sample = 25
display_bit = randint(size)
eve = 0

run(seed, size, sample, display_bit, eve)

Initial message qubit


┌───┐ ░ 
  q: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════

After Bob's measurement


┌───┐ ░ ┌─┐
  q: ┤ X ├─░─┤M├
     └───┘ ░ └╥┘
c: 1/═════════╩═
              0

alice_key =  [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0]
bob_key =  [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0]


...

┌───┐ ░ ┌───┐┌─┐
  q: ┤ X ├─░─┤ H ├┤M├
     └───┘ ░ └───┘└╥┘
c: 1/══════════════╩═
                   0

┌───┐ ░ ┌─┐
  q: ┤ X ├─░─┤M├
     └───┘ ░ └╥┘
c: 1/═════════╩═
              0

┌───┐ ░ ┌───┐┌─┐
  q: ┤ X ├─░─┤ H ├┤M├
     └───┘ ░ └───┘└╥┘
c: 1/══════════════╩═
                   0

┌───┐ ░ ┌───┐┌─┐
  q: ┤ H ├─░─┤ H ├┤M├
     └───┘ ░ └───┘└╥┘
c: 1/══════════════╩═
                   0

alice_sample = [0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]
  bob_sample = [0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]
Interception detected:  False


In [63]:
eve = 1

run(seed, size, sample, display_bit, eve)

Initial message qubit


┌───┐ ░ 
  q: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════

After Eve's measurement


┌───┐ ░ ┌─┐
  q: ┤ X ├─░─┤M├
     └───┘ ░ └╥┘
c: 1/═════════╩═
              0

After Bob's measurement


...

░ ┌─┐┌───┐┌─┐
  q: ─░─┤M├┤ H ├┤M├
      ░ └╥┘└───┘└╥┘
c: 1/════╩═══════╩═
         0       0

┌───┐ ░ ┌─┐┌─┐
  q: ┤ H ├─░─┤M├┤M├
     └───┘ ░ └╥┘└╥┘
c: 1/═════════╩══╩═
              0  0

░ ┌───┐┌─┐┌───┐┌─┐
  q: ─░─┤ H ├┤M├┤ H ├┤M├
      ░ └───┘└╥┘└───┘└╥┘
c: 1/═════════╩═══════╩═
              0       0

┌───┐ ░ ┌───┐┌─┐┌───┐┌─┐
  q: ┤ X ├─░─┤ H ├┤M├┤ H ├┤M├
     └───┘ ░ └───┘└╥┘└───┘└╥┘
c: 1/══════════════╩═══════╩═
                   0       0

alice_sample = [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0]
  bob_sample = [0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0]
Interception detected:  True
